In [ ]:
pip install qiskit
pip install qiskit-aer
pip install qiskit-machine-learning
pip install pylatexenc

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time

In [ ]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()

In [ ]:
df = pd.DataFrame(data =data.data, columns= data.feature_names)
df['target'] =data.target

In [ ]:
print("Total data points",df.shape[0])
print("Total number of features(as number of columns) are ", df.shape[1])
df.describe().transpose()

In [ ]:
df.groupby(["target"]).count()

In [ ]:
print("The unique number of data values are")
df.nunique()

In [ ]:
null_values = df.isnull().values.any()
if null_values == True:
    print("There are some missing values in data")
else:
    print("There are no missing values in the dataset")

In [ ]:
plt.figure(figsize =(30,30))
sns.heatmap(df.corr(), vmin=-1, vmax=1, annot=True)
plt.show()

In [ ]:
sns.set_theme(style="white")
corr = df.corr()
# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))
# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap,square=True, linewidths=.5, cbar_kws={"shrink": .6})

In [ ]:
d =df
X =d.drop(["target"], axis =1)
y=d.target

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X =pd.DataFrame(scaler.fit_transform(X), columns =X.columns)

In [ ]:
X.head()

In [ ]:
from sklearn.decomposition import PCA
pca= PCA(n_components = 4)
X =pca.fit_transform(X)
print(pca.explained_variance_ratio_)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
X = MinMaxScaler().fit_transform(X)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from qiskit.utils import algorithm_globals
algorithm_globals.random_seed =123
train_features, test_features, train_labels, test_labels = train_test_split(X, y, train_size=0.8, random_state=algorithm_globals.random_seed)

In [ ]:
from sklearn.svm import SVC
classical_kernels = ['rbf', 'linear', 'poly', 'sigmoid']

start =time.time()
for kernel in classical_kernels:
    classical_svc = SVC(kernel= kernel)
    cl_svc = classical_svc.fit(train_features, train_labels)
    y_svc = cl_svc.predict(test_features)
    svc_accuracy_score = accuracy_score(test_labels, y_svc)
    svc_precision_score = precision_score(test_labels, y_svc)
    svc_recall_score = recall_score(test_labels, y_svc)
    svc_f1_score = f1_score(test_labels, y_svc)
    print(f'{kernel} kernel classification accuracy score:  {svc_accuracy_score: .2f}')
    print(f'{kernel} kernel classification precision score:  {svc_precision_score: .2f}')
    print(f'{kernel} kernel classification recall score:  {svc_recall_score: .2f}')
    print(f'{kernel} kernel classification F1 score:  {svc_f1_score: .2f}')
    mat = confusion_matrix(test_labels,y_svc)
    sns.heatmap(mat.T, square=True, annot=True,fmt='d',cbar=False)
    plt.title(f'Confusion Matrix for {kernel} kernel')
    plt.xlabel('true class')
    plt.ylabel('predicted label')
    plt.show()
    print("**************")

elapsed = time.time() -start
print(f"Training and testing time: {elapsed} seconds")

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
start =time.time()
KNN = KNeighborsClassifier(n_neighbors = 13)
knn = KNN.fit(train_features, train_labels)
y_knn = knn.predict(test_features)
knn_accuracy_score = accuracy_score(test_labels, y_knn)
knn_precision_score = precision_score(test_labels, y_knn)
knn_recall_score = recall_score(test_labels, y_knn)
knn_f1_score = f1_score(test_labels, y_knn)
print(f'KNN classification accuracy score:  {knn_accuracy_score: .2f}')
print(f'KNN classification precsion score:  {knn_precision_score: .2f}')
print(f'KNN classification recall score:  {knn_recall_score: .2f}')
print(f'KNN classification F1 score:  {knn_f1_score: .2f}')
elapsed = time.time() -start
print(f"Training and testing time: {elapsed} seconds")

In [ ]:
mat = confusion_matrix(test_labels,y_knn)
sns.heatmap(mat.T, square=True, annot=True,fmt='d',cbar=False)
plt.title("Confusion Matrix for KNN")
plt.xlabel('true class')
plt.ylabel('predicted label')
plt.show()

In [ ]:
from qiskit.circuit.library import ZZFeatureMap

num_features = train_features.shape[1]

feature_map = ZZFeatureMap(feature_dimension=num_features, reps=3, entanglement = 'linear', insert_barriers =True)
feature_map.decompose().draw(output='mpl')


In [ ]:
from qiskit import Aer, execute
backend = Aer.get_backend("qasm_simulator")

In [ ]:
from qiskit_machine_learning.kernels import QuantumKernel
qunatum_kernel = QuantumKernel(feature_map=feature_map, quantum_instance=backend)

In [ ]:
quantum_svc = SVC(kernel=qunatum_kernel.evaluate)
start =time.time()
q = quantum_svc.fit(train_features, train_labels)
y_q = q.predict(test_features)
q_accuracy_score = accuracy_score(test_labels, y_q)
q_precision_score = precision_score(test_labels, y_q)
q_recall_score = recall_score(test_labels, y_q)
q_f1_score = f1_score(test_labels, y_q)
print(f" Callable quantum kernel classification accuracy score: {q_accuracy_score: .2f}")
print(f" Callable quantum kernel classification precision score: {q_precision_score: .2f}")
print(f" Callable quantum kernel classification recall score: {q_recall_score: .2f}")
print(f" Callable quantum kernel classification F1 score: {q_f1_score: .2f}")
elapsed = time.time() -start
print(f"Training and testing time: {elapsed} seconds")

In [ ]:
mat = confusion_matrix(test_labels,y_q)
sns.heatmap(mat.T, square=True, annot=True,fmt='d',cbar=False)
plt.title("Confusion Matrix for Quantum SVC")
plt.xlabel('true class')
plt.ylabel('predicted label')
plt.show()